In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
import numpy as np

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
weather_data_df = pd.read_csv("Weather_Data.csv")
weather_data_df.dtypes

City_ID              int64
City                object
Country             object
Date                object
Lat                float64
Lng                float64
Max Temp           float64
Humidity             int64
Cloudiness           int64
Current Weather     object
Rain(Inches)       float64
Snow(Inches)         int64
Wind Speed         float64
dtype: object

In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rainingYN = str(input("Do you want it to be raining?(yes/no)"))
snowingYN = str(input("Do you want it to be snowing?(yes/no)"))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 100
Do you want it to be raining?(yes/no)no
Do you want it to be snowing?(yes/no)no


In [5]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & \
                                          (weather_data_df["Max Temp"] >= min_temp)]
if rainingYN == "no" and snowingYN == "no":
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain(Inches)"] == 0) & \
                                                  (preferred_cities_df["Snow(Inches)"] == 0)]
elif rainingYN == "yes" and snowingYN == "yes":
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain(Inches)"] > 0) & \
                                                  (preferred_cities_df["Snow(Inches)"] > 0)]
elif rainingYN == "yes" and snowingYN == "no":
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain(Inches)"] > 0) & \
                                                  (preferred_cities_df["Snow(Inches)"] == 0)]
elif rainingYN == "no" and snowingYN == "yes":
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain(Inches)"] == 0) & \
                                                  (preferred_cities_df["Snow(Inches)"] > 0)]    
    
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Current Weather,Rain(Inches),Snow(Inches),Wind Speed
2,2,Pangai,TO,2020-07-13 02:53:46,-19.80,-174.35,75.20,78,75,broken clouds,0.0,0,18.34
7,7,Pringsewu,ID,2020-07-13 02:53:47,-5.36,104.97,86.14,58,76,broken clouds,0.0,0,2.33
14,14,Belogorsk,RU,2020-07-13 02:53:47,50.92,128.46,85.32,54,1,clear sky,0.0,0,6.20
15,15,Beidao,CN,2020-07-13 02:53:47,34.57,105.89,79.95,49,100,overcast clouds,0.0,0,3.33
23,23,Salalah,OM,2020-07-13 02:53:48,17.02,54.09,80.60,100,90,drizzle,0.0,0,4.70
26,26,Labuhan,ID,2020-07-13 02:53:48,-6.88,112.21,83.16,67,74,broken clouds,0.0,0,5.86
27,27,Hirara,JP,2020-07-13 02:53:48,24.80,125.28,91.40,63,75,broken clouds,0.0,0,8.05
35,35,Albany,US,2020-07-13 02:50:10,42.60,-73.97,77.00,90,79,broken clouds,0.0,0,4.34
37,37,Kavaratti,IN,2020-07-13 02:53:49,10.57,72.64,83.23,75,100,overcast clouds,0.0,0,11.12
42,42,Balkanabat,TM,2020-07-13 02:53:50,39.51,54.37,84.36,39,62,broken clouds,0.0,0,5.59


In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Weather", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.count()

City               166
Country            166
Max Temp           166
Current Weather    166
Lat                166
Lng                166
Hotel Name         166
dtype: int64

In [7]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [8]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found, skipped")
hotel_df.count()

Hotel not found, skipped
Hotel not found, skipped
Hotel not found, skipped
Hotel not found, skipped
Hotel not found, skipped
Hotel not found, skipped
Hotel not found, skipped
Hotel not found, skipped
Hotel not found, skipped
Hotel not found, skipped
Hotel not found, skipped
Hotel not found, skipped
Hotel not found, skipped


City               166
Country            166
Max Temp           166
Current Weather    166
Lat                166
Lng                166
Hotel Name         166
dtype: int64

In [15]:
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
2,Pangai,TO,75.20,broken clouds,-19.80,-174.35,Ha'apai Beach Resort
7,Pringsewu,ID,86.14,broken clouds,-5.36,104.97,Urban Style by Front One
14,Belogorsk,RU,85.32,clear sky,50.92,128.46,Hotel Malina
15,Beidao,CN,79.95,overcast clouds,34.57,105.89,Shell Tianshui Maiji District Erma Road Walkin...
23,Salalah,OM,80.60,drizzle,17.02,54.09,HAMDAN PLAZA HOTEL
...,...,...,...,...,...,...,...
548,Cayenne,GF,75.20,few clouds,4.93,-52.33,Hôtel Le Dronmi
549,Athens,GR,78.80,clear sky,37.98,23.72,Novotel Athenes
550,Verkhoyansk,RU,81.93,few clouds,67.54,133.38,NaN
551,Saint-Louis,SN,82.99,scattered clouds,16.33,-15.00,NaN


In [17]:
hotel_df.dropna()

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
2,Pangai,TO,75.20,broken clouds,-19.80,-174.35,Ha'apai Beach Resort
7,Pringsewu,ID,86.14,broken clouds,-5.36,104.97,Urban Style by Front One
14,Belogorsk,RU,85.32,clear sky,50.92,128.46,Hotel Malina
15,Beidao,CN,79.95,overcast clouds,34.57,105.89,Shell Tianshui Maiji District Erma Road Walkin...
23,Salalah,OM,80.60,drizzle,17.02,54.09,HAMDAN PLAZA HOTEL
...,...,...,...,...,...,...,...
544,Karaikkudi,IN,84.47,overcast clouds,10.07,78.78,Hotel Meyyappaa
545,Trat,TH,80.60,scattered clouds,12.50,102.50,สวนหนิง-อัศ
548,Cayenne,GF,75.20,few clouds,4.93,-52.33,Hôtel Le Dronmi
549,Athens,GR,78.80,clear sky,37.98,23.72,Novotel Athenes


In [18]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [21]:
# Add for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")